In [13]:
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, Flatten, MaxPooling2D, Input, BatchNormalization, Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import vgg16

In [14]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
 tf.config.experimental.set_memory_growth(gpu, True)

In [15]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [16]:
train_path = 'train'
validation_path = 'val'
test_path = 'test'

def datagen_function(path, aug=False):
    if aug == True:
        generator = ImageDataGenerator(
            rescale = 1 / 255.0,
            rotation_range = 10,
            zoom_range = 0.1,
            horizontal_flip = True
        )
    else:
        generator = ImageDataGenerator(
            rescale = 1 / 255.0
        )
    return generator.flow_from_directory(
        path,
        target_size = (48,48),
        color_mode = 'rgb',
        shuffle = True,
        class_mode = 'categorical',
        batch_size = 64
    )

In [17]:
train_gen = datagen_function(train_path, True)
validation_gen = datagen_function(validation_path)
test_gen = datagen_function(test_path)

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [22]:
model = Sequential()

pretrained_model = vgg16.VGG16(include_top=False, input_shape=(48, 48, 3),classes=7, weights='imagenet', pooling = 'max')
for layer in pretrained_model.layers:
       layer.trainable=False

model.add(pretrained_model)
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(7, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.0005),loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten_2 (Flatten)         (None, 512)               0         
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 7)                 3591      
                                                                 
Total params: 14,718,279
Trainable params: 3,591
Non-trainable params: 14,714,688
_________________________________________________________________


In [1]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.5, patience = 4, verbose = 1)
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

model.compile(loss = tf.keras.losses.categorical_crossentropy, metrics = 'acc', optimizer = optimizer)
history = model.fit(train_gen,validation_data = validation_gen, epochs = 100, callbacks = [reduce_lr])

NameError: name 'ReduceLROnPlateau' is not defined